In [248]:
import numpy as np
from dataclasses import dataclass
from functools import cache
from itertools import zip_longest
import sys

@dataclass
class Piece():
  rows: np.ndarray[np.int8, np.int8]
  columns: np.ndarray[np.int8, np.int8]

  def __repr__(self):
    buffer = []
    for row, column in zip_longest(self.rows, self.columns):
        if row is not None:
          buffer.append(" " + " ".join('-' if elem else ' ' for elem in row))
        if column is not None:
          buffer.append(" ".join('|' if elem else ' ' for elem in column))
    return "\n".join(buffer)
  
  def __hash__(self):
     return hash((str(self.rows), str(self.columns)))

  def __eq__(self, other):
      if isinstance(other, Piece):
          return np.array_equal(self.rows, other.rows) and np.array_equal(self.columns, other.columns)
      return NotImplemented
    
  def rotated(self):
    rows = np.rot90(self.columns, 1)
    columns = np.rot90(self.rows, 1)

    while ~np.any(rows[0]) and ~np.any(columns[0]):
      # top row is empty, so we can shift digit up
      rows = np.roll(rows, -1, 0)
      columns = np.roll(columns, -1, 0)

    return Piece(rows, columns)
  
  @cache
  def trimmed(self):
    rows = self.rows
    columns = self.columns

    while rows.size and ~np.any(rows[-1]) and columns.size and ~np.any(columns[-1]):
       rows = rows[:-1]
       columns = columns[:-1]

    while rows.size and ~np.any(rows[:,-1]) and columns.size and ~np.any(columns[:,-1]):
       rows = rows[:,:-1]
       columns = columns[:,:-1]
    
    return Piece(rows, columns)

  @cache
  def rotations(self):
    a = self.rotated()
    b = a.rotated()
    c = b.rotated()
    d = c.rotated()
    return set((a, b, c, d))

pieces = [
  Piece(rows=np.asarray(((True, False), (True, False), (False, False)), dtype=np.int8), columns=np.asarray(((True, True, False), (False, False, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((False, False), (False, False), (False, False)), dtype=np.int8), columns=np.asarray(((True, False, False), (True, False, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((True, False), (True, False), (True, False)), dtype=np.int8), columns=np.asarray(((False, True, False), (True, False, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((True, False), (True, False), (True, False)), dtype=np.int8), columns=np.asarray(((False, True, False), (False, True, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((False, False), (True, False), (False, False)), dtype=np.int8), columns=np.asarray(((True, True, False), (False, True, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((True, False), (True, False), (True, False)), dtype=np.int8), columns=np.asarray(((True, False, False), (False, True, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((True, False), (True, False), (True, False)), dtype=np.int8), columns=np.asarray(((True, False, False), (True, True, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((True, False), (False, False), (False, False)), dtype=np.int8), columns=np.asarray(((False, True, False), (False, True, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((True, False), (True, False), (True, False)), dtype=np.int8), columns=np.asarray(((True, True, False), (True, True, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((True, False), (True, False), (True, False)), dtype=np.int8), columns=np.asarray(((True, True, False), (False, True, False)), dtype=np.int8)),
]

print(sys.getsizeof(pieces[0].rows))

134


In [249]:
pieces[0]

 -  
| |  
 -  
     
    

In [250]:
pieces[1].trimmed()

 
|
 
|
 

In [251]:
pieces[9].rotations()

{   -
 | | |
  - -
      
     ,
  -  
 |    
  -  
 | |  
  -  ,
  -  
 | |  
  -  
   |  
  -  ,
  - -
 | | |
  -  
      
     }

In [252]:
pieces[0].rotations()

{ -  
 | |  
  -  
      
     }

In [253]:
class Puzzle():
  rows: np.ndarray[np.int8, np.int8]
  cols: np.ndarray[np.int8, np.int8]

  def __init__(
      self,
      rows=np.zeros((5,5), dtype=np.int8),
      cols=np.zeros((4,6), dtype=np.int8),
    ):
    self.rows = rows
    self.cols = cols

  def __repr__(self):
    buffer = []
    for row, column in zip_longest(self.rows, self.cols):
        if row is not None:
          buffer.append(" " + " ".join('-' if elem else '.' for elem in row))
        if column is not None:
          buffer.append(" ".join('|' if elem else '.' for elem in column))
    return "\n".join(buffer)

  def __hash__(self):
    return hash((str(self.rows), str(self.cols)))

  def __eq__(self, other):
    if isinstance(other, Puzzle):
        return np.array_equal(self.rows, other.rows) and np.array_equal(self.cols, other.cols)
    return NotImplemented
  
  @cache
  def with_piece(self, piece: Piece, toprow: int, topcol: int) -> "Puzzle":
    piece = piece.trimmed()

    if (toprow + piece.rows.shape[0] > self.rows.shape[0]):
      return None
    if (topcol + piece.rows.shape[1] > self.rows.shape[1]):
      return None
    if (toprow + piece.columns.shape[0] > self.cols.shape[0]):
      return None
    if (topcol + piece.columns.shape[1] > self.cols.shape[1]):
      return None
    
    puzzle_rows = self.rows[toprow:toprow+piece.rows.shape[0], topcol:topcol+piece.rows.shape[1]]
    piece_rows_nonzero = np.nonzero(piece.rows)
    if np.any(puzzle_rows[piece_rows_nonzero]):
      return None

    puzzle_cols = self.cols[toprow:toprow+piece.columns.shape[0], topcol:topcol+piece.columns.shape[1]]
    piece_cols_nonzero = np.nonzero(piece.columns)
    if np.any(puzzle_cols[piece_cols_nonzero]):
      return None
    
    new_rows = self.rows.copy()
    new_cols = self.cols.copy()
    puzzle_rows = new_rows[toprow:toprow+piece.rows.shape[0], topcol:topcol+piece.rows.shape[1]]
    puzzle_cols = new_cols[toprow:toprow+piece.columns.shape[0], topcol:topcol+piece.columns.shape[1]]
    puzzle_rows[piece_rows_nonzero] = piece.rows[piece_rows_nonzero]
    puzzle_cols[piece_cols_nonzero] = piece.columns[piece_cols_nonzero]

    return Puzzle(new_rows, new_cols)
  
  def spaces_left(self):
    return np.size(self.cols) - np.count_nonzero(self.cols) + np.size(self.rows) - np.count_nonzero(self.rows)

puzzle = Puzzle()

In [254]:
%timeit Puzzle().with_piece(pieces[7], 3, 4)

568 µs ± 89.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [255]:
%timeit Puzzle().spaces_left()

4.26 µs ± 701 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [256]:
Puzzle()

 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [257]:
puzzle.spaces_left()

49

In [258]:
puzzle.with_piece(pieces[0], 1, 4)

 . . . . .
. . . . . .
 . . . . -
. . . . | |
 . . . . -
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [259]:
puzzle.with_piece(pieces[7].rotated(), 0, 0)

 - - . . .
| . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [260]:
puzzle.with_piece(pieces[3], 0, 0).with_piece(pieces[4].rotated(), 0, 1)

 - - - . .
. | | . . .
 - - . . .
. | . . . .
 - . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [261]:
Puzzle().with_piece(pieces[9].rotated().rotated().rotated(), 1, 0)

 . . . . .
. . . . . .
 . - . . .
| | | . . .
 - - . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [263]:
from copy import copy

bad_puzzles = set()

def attempt(puzzle: Puzzle, used_pieces: tuple):
  if puzzle in bad_puzzles:
    return None
  if puzzle.spaces_left() == 2:
    return puzzle, used_pieces
  
  candidates = []
  for i in range(len(pieces)):
    if i in [j for (j, _, _, _) in used_pieces]:
      continue

    for rotation in pieces[i].rotations():
      for row in range(0, len(puzzle.rows)):
        for col in range(0, len(puzzle.cols)):
          candidate = puzzle.with_piece(rotation, row, col)
    
          if candidate is not None:
            if candidate in bad_puzzles:
              continue

            candidate_used_pieces = copy(used_pieces)
            candidate_used_pieces.append((i, rotation, row, col))
            candidates.append((candidate, candidate_used_pieces))
    
  for candidate, candidate_used_pieces in candidates:
    result = attempt(candidate, candidate_used_pieces)
    if result is not None:
      return result

  bad_puzzles.add(puzzle)
  return None

# pick a puzzle piece
# choose rotation
# find top-left free corner
# put piece if it fits
# check for solved
rot9 = pieces[9].rotated().rotated().rotated()
puzzle = Puzzle().with_piece(rot9, 1, 0)
attempted = attempt(puzzle, [(9, rot9, 1, 0)])
attempted

KeyboardInterrupt: 

In [ ]:
len(bad_puzzles)

215377

In [ ]:
next(iter(bad_puzzles))

 - - - - .
| | . | . .
 - - - - .
| | | . . .
 - - - - .
. . | . . .
 - - . . .
| | | . . .
 - . . . .